<a href="https://colab.research.google.com/github/ahmededrisahmed/fmcs39130/blob/main/updateCodeLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**First we will start with the imports:**

In [ ]:
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import io
import re
from nltk.stem import PorterStemmer

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [ ]:
from google.colab import files 
uploaded = files.upload()

In [ ]:
# data = pd.read_csv('amazon_alexa.tsv')

In [ ]:
# Now use the below code to read the csv in pandas' dataframe
data = pd.read_csv(io.StringIO(uploaded['amazon_alexa.tsv'].decode('utf-8')), sep='\t')
data = data[data.verified_reviews != "Neutral"]
data = data[['verified_reviews', 'feedback']]
# if(data['feedback'] == 1).any():
#   print(data['verified_reviews'])

In [ ]:
def cleanData(fiald, feedback):
    data[fiald] = data[fiald].apply(lambda x: x.lower())
    data[fiald] = data[fiald].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

    print(data[ data[feedback] == 1 ].size)
    print(data[ data[feedback] == 0 ].size)

    for idx,row in data.iterrows():
      row[0] = row[0].replace('rt',' ')
      
cleanData('verified_reviews', 'feedback')


In [ ]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['verified_reviews'].values)
X = tokenizer.texts_to_sequences(data['verified_reviews'].values)
X = pad_sequences(X)
print(X)

In [ ]:
Y = pd.get_dummies(data['feedback']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
# print(X_train.shape,Y_train.shape)
# print(X_test.shape,Y_test.shape)
print(Y_test)
print(X_test)

In [ ]:
embed_dim = 128
lstm_out = 196
max_fatures = 2000
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

In [ ]:
batch_size = 32
history = model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)

In [ ]:
# batch_size = 32
# model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)

In [ ]:
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history.history) 
hist_df

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
y_pred = model.predict(X_test)
print(y_pred)
# s = []
# for i in range(y_pred):
#   if(y_pred[i] >= 0.5).all():
#     s.append(1)
#   else:
#     s.append(0)

# print(s)

In [ ]:

# import modules
import pandas as pd
import tweepy

# function to display data of each tweet
def printtweetdata(n, ith_tweet):
		print()
		print(f"Tweet {n}:")
		print(f"Username:{ith_tweet[0]}")
		print(f"Description:{ith_tweet[1]}")
		print(f"Location:{ith_tweet[2]}")
		print(f"Following Count:{ith_tweet[3]}")
		print(f"Follower Count:{ith_tweet[4]}")
		print(f"Total Tweets:{ith_tweet[5]}")
		print(f"Retweet Count:{ith_tweet[6]}")
		print(f"Tweet Text:{ith_tweet[7]}")
		print(f"Hashtags Used:{ith_tweet[8]}")


# function to perform data extraction
def scrape(words, date_since, numtweet):

		# Creating DataFrame using pandas
		db = pd.DataFrame(columns=['username',
								'description',
								'location',
								'following',
								'followers',
								'totaltweets',
								'retweetcount',
								'text',
								'hashtags'])

		# We are using .Cursor() to search
		# through twitter for the required tweets.
		# The number of tweets can be
		# restricted using .items(number of tweets)
		tweets = tweepy.Cursor(api.user_timeline,
							words, lang="en",
							since_id=date_since,
							tweet_mode='extended').items(numtweet)


		# .Cursor() returns an iterable object. Each item in
		# the iterator has various attributes
		# that you can access to
		# get information about each tweet
		list_tweets = [tweet for tweet in tweets]

		# Counter to maintain Tweet Count
		i = 1

		# we will iterate over each tweet in the
		# list for extracting information about each tweet
		for tweet in list_tweets:
				username = tweet.user.screen_name
				description = tweet.user.description
				location = tweet.user.location
				following = tweet.user.friends_count
				followers = tweet.user.followers_count
				totaltweets = tweet.user.statuses_count
				retweetcount = tweet.retweet_count
				hashtags = tweet.entities['hashtags']

				# Retweets can be distinguished by
				# a retweeted_status attribute,
				# in case it is an invalid reference,
				# except block will be executed
				try:
						text = tweet.retweeted_status.full_text
				except AttributeError:
						text = tweet.full_text
				hashtext = list()
				for j in range(0, len(hashtags)):
						hashtext.append(hashtags[j]['text'])

				# Here we are appending all the
				# extracted information in the DataFrame
				ith_tweet = [username, description,
							location, following,
							followers, totaltweets,
							retweetcount, text, hashtext]
				db.loc[len(db)] = ith_tweet

				# Function call to print tweet data on screen
				printtweetdata(i, ith_tweet)
				i = i+1
		filename = 'scraped_tweets.csv'

		# we will save our database as a CSV file.
		db.to_csv(filename)
  

if __name__ == '__main__':

		# Enter your own credentials obtained
		# from your developer account
		consumer_key = "sDq4iViNrUs01nRtozqjlFMXC"
		consumer_secret = "AhMHrrq8qYt4aAueIKiwxjLI2SRHZ4vpI84UMXS8IEojpdCsxt"
		access_key = "1588248654548279301-RglNooFZz3LgM5BLCmtNLN0mdS1wpz"
		access_secret = "uUx5tsnNqgb3gyFqhLaUjlEAE3YCUYl4rLPQVssYkvEUJ"


		auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
		auth.set_access_token(access_key, access_secret)
		api = tweepy.API(auth)

		# Enter Hashtag and initial date
		print("Enter Twitter HashTag to search for")
		words = input()
		print("Enter Date since The Tweets are required in yyyy-mm--dd")
		date_since = input()

		# number of tweets you want to extract in one run
		numtweet = 100
		scrape(words, date_since, numtweet)
		print('Scraping has completed!')


In [ ]:
new_data = pd.read_csv('scraped_tweets.csv')
new_data['text']

In [ ]:
# tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(new_data['text'].values)
final = tokenizer.texts_to_sequences(new_data['text'].values)
final = pad_sequences(final)
print(final)

In [ ]:
file_review = "review.pkl"

In [ ]:
# save model
import pickle
pickle.dump(model ,open(file_review,"wb"))

In [ ]:
# Load the trained model
with open(file_review, "rb") as file:
  pic_var = pickle.load(file)

In [ ]:
pic_var

In [ ]:
# Make new prediction
new_predict = pic_var.predict(final)